In [1]:
import pandas as pd
import numpy as np
import os

print("--- Menjalankan Skrip V10 (Persiapan Data untuk V12) ---")

# Muat & Transformasi Data ---
try:
    df = pd.read_csv("tokopedia.csv")
    df['order_date'] = pd.to_datetime(df['order_date'])
    df['total_sales'] = df['after_discount']
    df['year'] = df['order_date'].dt.year
    df['month_year'] = df['order_date'].dt.to_period('M').astype(str)
    df['month_num'] = df['order_date'].dt.month
    print("Transformasi data berhasil.")
except Exception as e:
    print(f"ERROR saat transformasi data: {e}")
    exit()

# Agregasi Data CSV/JSON untuk Dashboard ---
print("\n--- 2. Membuat Agregasi Data V10/V12 ---")
try:
    
    # 1. KPI Cards
    total_penjualan = df['total_sales'].sum()
    total_transaksi = len(df)
    aov = df['total_sales'].mean()
    kategori_top = df.groupby('category')['total_sales'].sum().idxmax()
    pd.DataFrame([
        {'metric': 'Total Penjualan', 'value': f"IDR {total_penjualan:,.0f}"},
        {'metric': 'Total Transaksi', 'value': f"{total_transaksi:,.0f}"},
        {'metric': 'Avg. Order Value', 'value': f"IDR {aov:,.0f}"},
        {'metric': 'Kategori Teratas', 'value': kategori_top}
    ]).to_csv('kpi_metrics.csv', index=False)
    print("File 'kpi_metrics.csv' dibuat.")

    # 2. Insight Boxes
    sales_2021 = df[df['year'] == 2021]['total_sales'].sum()
    sales_2022 = df[df['year'] == 2022]['total_sales'].sum()
    sales_growth_pct = ((sales_2022 - sales_2021) / sales_2021) * 100
    cod_pct = (df['payment_method'].value_counts(normalize=True)['cod']) * 100
    top_15_sku_names = df.groupby('sku_name')['total_sales'].sum().nlargest(15).index
    samsung_count = top_15_sku_names.str.contains('Samsung', case=False).sum()
    pd.DataFrame([
        {'insight': 'Pertumbuhan Penjualan (YoY)', 'value': f"{sales_growth_pct:.1f}%", 'narrative': 'Pertumbuhan dari 2021 ke 2022'},
        {'insight': 'Dominasi Metode COD', 'value': f"{cod_pct:.1f}%", 'narrative': 'dari total transaksi'},
        {'insight': 'Produk Samsung', 'value': f"{samsung_count} dari 15", 'narrative': 'masuk dalam Top 15 Produk Terlaris'}
    ]).to_csv('insight_metrics.csv', index=False)
    print("File 'insight_metrics.csv' dibuat.")

    # 3. Tren Penjualan (Line Chart)
    sales_trend = df.groupby(['month_year', 'year', 'month_num'])['total_sales'].sum().reset_index()
    sales_trend = sales_trend.sort_values(by=['year', 'month_num'])
    sales_trend['pct_change'] = (sales_trend['total_sales'].pct_change() * 100).fillna(0)
    max_sales_idx = sales_trend['total_sales'].idxmax()
    min_sales_val = sales_trend['total_sales'].nsmallest(2).iloc[1]
    min_sales_idx = sales_trend[sales_trend['total_sales'] == min_sales_val].index[0]
    sales_trend['annotation'] = ''
    sales_trend.loc[max_sales_idx, 'annotation'] = 'Puncak Penjualan'
    sales_trend.loc[min_sales_idx, 'annotation'] = 'Penurunan Terendah'
    sales_trend.to_csv('tren_penjualan_bulanan.csv', index=False)
    print("File 'tren_penjualan_bulanan.csv' dibuat.")

    # 4. Kategori (Bar Chart)
    top_10_cat = df.groupby('category')['total_sales'].sum().nlargest(10).sort_values(ascending=True).reset_index()
    top_10_cat.to_csv('top_10_kategori_barchart.csv', index=False)
    print("File 'top_10_kategori_barchart.csv' dibuat.")

    # 5. Metode Pembayaran (Pie Chart)
    payment_counts = df['payment_method'].value_counts()
    top_5_payment = payment_counts.nlargest(5).reset_index()
    top_5_payment.columns = ['payment_method', 'count']
    others_count = payment_counts.iloc[5:].sum()
    others_names = payment_counts.iloc[5:].index.tolist()
    if others_count > 0:
        others_df = pd.DataFrame([{'payment_method': 'Lainnya', 'count': others_count}])
        pie_data = pd.concat([top_5_payment, others_df], ignore_index=True)
    else:
        pie_data = top_5_payment
    pie_data.to_csv('pie_chart_payment.csv', index=False)
    print("File 'pie_chart_payment.csv' dibuat.")
    pd.DataFrame({'metode_lainnya': others_names}).to_csv('pie_chart_payment_others.csv', index=False)
    print("File 'pie_chart_payment_others.csv' dibuat.")

    # 6. Top 15 Produk (JSON)
    top_15_sku = df.groupby('sku_name')['total_sales'].sum().nlargest(15).sort_values(ascending=True).reset_index()
    top_15_sku['rank'] = range(len(top_15_sku), 0, -1)
    
    # File 1: Untuk Chart (Hanya data esensial)
    top_15_sku['sku_name_short'] = top_15_sku['sku_name'].str.slice(0, 40) # Nama untuk hover
    chart_data = top_15_sku[['rank', 'sku_name_short', 'total_sales']]
    
    with open('top_15_products.json', 'w') as f:
        f.write(chart_data.to_json(orient='records'))
    print("File 'top_15_products.json' (V10) dibuat.")
        
    print("\nSemua file untuk V12 berhasil dibuat.")

except Exception as e:
    print(f"ERROR saat agregasi data: {e}")
    exit()

--- Menjalankan Skrip V10 (Persiapan Data untuk V12) ---
Transformasi data berhasil.

--- 2. Membuat Agregasi Data V10/V12 ---
File 'kpi_metrics.csv' dibuat.
File 'insight_metrics.csv' dibuat.
File 'tren_penjualan_bulanan.csv' dibuat.
File 'top_10_kategori_barchart.csv' dibuat.
File 'pie_chart_payment.csv' dibuat.
File 'pie_chart_payment_others.csv' dibuat.
File 'top_15_products.json' (V10) dibuat.

Semua file untuk V12 berhasil dibuat.
